# Transduction from observation to prediction

### Introduction

Excerpt from https://en.wikipedia.org/wiki/Transduction_(machine_learning)):

In logic, statistical inference, and supervised learning, transduction or transductive inference is reasoning from observed, specific (training) cases to specific (test) cases. In contrast, induction is reasoning from observed training cases to general rules, which are then applied to the test cases. The distinction is most interesting in cases where the predictions of the transductive model are not achievable by any inductive model. Note that this is caused by transductive inference on different test sets producing mutually inconsistent predictions.

In [1]:
# Run the observation_prediction script
# It requires Python3, tensorflow and tensor2tensor.

! ./observation_prediction.sh

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Importing user module transduction from path /home/jdroo/github.com/josd/eye
[2018-04-24 11:37:20,223] Importing user module transduction from path /home/jdroo/github.com/josd/eye
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction
[2018-04-24 11:37:20,224] Generating problems:
    observation:
      * observation_prediction
INFO:tensorflow:Generating data for observation_prediction.
[2018-04-24 11:37:20,225] Generating data for observation_prediction.
INFO:tensorflow:Found vocab file: /home/jdroo/t2t_data/vocab.observation_prediction.16384.subwords
[2018-04-24 11:37:

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_754_256.bottom
[2018-04-24 11:37:31,869] Transforming feature 'inputs' with symbol_modality_754_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_754_256.targets_bottom
[2018-04-24 11:37:32,071] Transforming 'targets' with symbol_modality_754_256.targets_bottom
INFO:tensorflow:Building model body
[2018-04-24 11:37:32,080] Building model body
Instructions for updating:
keep_dims is deprecated, use keepdims instead
[2018-04-24 11:37:32,172] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/layers/common_layers.py:553: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Transforming body output with symbol_modality_754_256.top
[2018-04-24 11:37:34,251] Transforming body output with symbol_modality_754_256.top
Instructions for updati

INFO:tensorflow:Running local_init_op.
[2018-04-24 11:38:55,601] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-04-24 11:38:55,652] Done running local_init_op.
INFO:tensorflow:Evaluation [1/3]
[2018-04-24 11:38:56,880] Evaluation [1/3]
INFO:tensorflow:Evaluation [2/3]
[2018-04-24 11:38:57,143] Evaluation [2/3]
INFO:tensorflow:Evaluation [3/3]
[2018-04-24 11:38:57,370] Evaluation [3/3]
INFO:tensorflow:Finished evaluation at 2018-04-24-09:38:57
[2018-04-24 11:38:57,466] Finished evaluation at 2018-04-24-09:38:57
INFO:tensorflow:Saving dict for global step 2500: global_step = 2500, loss = 0.03959937, metrics-observation_prediction/accuracy = 0.9957976, metrics-observation_prediction/accuracy_per_sequence = 0.97414964, metrics-observation_prediction/accuracy_top5 = 0.998721, metrics-observation_prediction/approx_bleu_score = 0.7544753, metrics-observation_prediction/neg_log_perplexity = -0.09629734, metrics-observation_prediction/rouge_2_fscore = 0.82804865, metri

Instructions for updating:
keep_dims is deprecated, use keepdims instead
[2018-04-24 11:39:12,593] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/utils/beam_search.py:93: calling reduce_logsumexp (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Done calling model_fn.
[2018-04-24 11:39:12,674] Done calling model_fn.
INFO:tensorflow:Graph was finalized.
[2018-04-24 11:39:12,867] Graph was finalized.
2018-04-24 11:39:12.868217: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-04-24 11:39:12.977514: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-04-24 11:39:12.978021: I tensorflow/core/

In [2]:
# See the transductions

! pygmentize -g sample.observation
print("->-")
! pygmentize -g sample.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N	-0.37	A_PERSON has BMI class U	-4.58	A_PERSON has BMI class O	-4.83
A_TURBINE producing 9533 kW	-0.37	A_TURBINE producing 16682 kW	-3.19	A_TURBINE producing 2383 kW	-3.54
